In [1]:
import re
from pathlib import Path
import pandas as pd

In [2]:
data_dir = Path("./data")
email_df = pd.read_csv(data_dir / "mail_samples_7th_Apr_24.csv")
email_df.head()

,subject,from,body,sent_files,date
0,Check out New Jobs Posted by Top Companies,"""iimjobs.com"" <info@iimjobs.com>",Please Enable Javascript\n<!DOCTYPE html> <htm...,NaN,2024-03-31 16:34:37+00:00
1,lunarflu mentioned you in Hugging Face,Discord <notifications@discord.com>,"\r\n <!doctype html>\r\n <html xmlns=""ht...",NaN,2024-03-31 15:50:21+00:00
2,"""Comparing with Apple.... too much""",Reddit <noreply@redditmail.com>,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 S...",NaN,2024-03-31 15:44:47+00:00
3,Suprriya Kauul ( She / Her / Hers) sent you a ...,LinkedIn <messages-noreply@linkedin.com>,-----------------------------------------\r\n\...,NaN,2024-03-31 13:21:48+00:00
4,Rina (Owner) has a Residential House in Kaikh...,Magicbricks <projectsonmb@magicbricks.com>,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",NaN,2024-03-31 18:47:30+05:30


In [4]:
email_types, senders = [], []
missed = []
for v in email_df["from"].str.strip().values:
    buffer = re.findall(r"\<(.*)@(.*)\.com\>", v)
    if len(buffer):
        email_types.append(buffer[0][0])
        senders.append(buffer[0][1])
    else:
        missed.append(v)

In [5]:
# ## email sender and mail type analysis
# import json
# from collections import defaultdict

# tmp = defaultdict(lambda: [])
# for v1, v2 in zip(email_types, senders):
#     tmp[v2].append(v1)
# tmp = {k: list(set(tmp[k])) for k in tmp}
# with open("tmp.json", "w") as file:
#     json.dump(tmp, file)

In [15]:
missed = list(set(missed))
len(missed)

33

In [54]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# model = "gemma:7b"
model_name = "openhermes:latest"
model = ChatOllama(model=model_name)

In [104]:
from langchain_core.pydantic_v1 import BaseModel, Field, validator, Extra
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from langchain_core.output_parsers import JsonOutputParser


class OutEntities(BaseModel):
    type: str = Field(description="a noun/action-word from the input")
    sender: str = Field(
        description="a noun word without any '.in'/'ai'/'co.in' etc, which will be stripped"
    )

    class Config:
        extra = Extra.forbid

    # # You can add custom validation logic easily with Pydantic.
    # @validator("setup")
    # def question_ends_with_question_mark(cls, field):
    #     if field[-1] != "?":
    #         raise ValueError("Badly formed question!")
    #     return field

In [105]:
task = f"""
# Task
You have to extract entities from given 

- alexey@datatalks.club
"""

explanation = """
Keep output verbose, only return the format given.
For the case: account-update@amazon.in
you extract: {"type": "account-update", "sender": "amazon"}

In the above case you took the noun/action-item as "type"
and "sender" which is a noun without any ".in"/"ai"/"co.in" etc

Based on the above case, do the same for the below input
"""

query = f"{explanation}{task}"
print(query)


Keep output verbose, only return the format given.
For the case: account-update@amazon.in
you extract: {"type": "account-update", "sender": "amazon"}

In the above case you took the noun/action-item as "type"
and "sender" which is a noun without any ".in"/"ai"/"co.in" etc

Based on the above case, do the same for the below input

# Task
You have to extract entities from given 

- alexey@datatalks.club



In [70]:
from langchain.output_parsers import RetryOutputParser, RegexDictParser
from langchain_core.runnables import RunnableLambda, RunnableParallel

In [117]:
parser = PydanticOutputParser(pydantic_object=OutEntities)
prompt = PromptTemplate(
    template="""Answer the user query.\n{format_instructions}\n{query}\n""",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

completion_chain = prompt | model

In [118]:
resp = completion_chain.invoke({"query":query})

AIMessage(content='For the case: alexey@datatalks.club\nwe extract: {"type": "alexey", "sender": "datatalks"}\n\nIn this case, we took the noun/action-item as "type" and "sender" which is a noun without any ".in"/"ai"/"co.in" etc.', response_metadata={'model': 'openhermes:latest', 'created_at': '2024-04-07T18:23:42.094867Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 10386806291, 'load_duration': 629871375, 'prompt_eval_count': 381, 'prompt_eval_duration': 3537110000, 'eval_count': 79, 'eval_duration': 6216636000}, id='run-f37f2a31-4c79-4cd0-bfca-da097d32d265-0')

In [123]:
import json
match = re.search("\{.*\}", resp.content)
if match:
    print(json.loads(match.group(0)))

{'type': 'email', 'sender': 'alexey'}


In [120]:
match

<re.Match object; span=(86, 123), match='{"type": "email", "sender": "alexey"}'>

In [12]:
# with open("tmp.txt", "a") as file:
#     for v1, v2 in zip(email_types, senders):
#         file.write(f"{v1} | {v2}")
#         file.write("\n")

### Parse Mail sender and action type from sender email

In [ ]:
email_type, sender = re.findall(
    r"\<(.*)@(.*)\.com\>", '"iimjobs.com" <info@iimjobs.com>'
)[0]

email_df["from"].apply(lambda x: re.search(r"\<(.*)@\(.*)\.com>")).value_counts()

In [ ]:
re.sub("\W", "", email_df["subject"][6])

In [ ]:
resp = llm(
    f"""
    INPUT:
    {email_df["subject"][6]}
    
    TASK: Extract only the english text from the given sentence, keep output verbose
    OUTPUT: Use the output format: 

    english_text: // place the text here
    """
)

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser


class EnglishText(BaseModel):
    english_text: str


class OutEntities(BaseModel):
     = Field(description="")
    
    
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


In [ ]:
parser = PydanticOutputParser(pydantic_object=EnglishText)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt_and_model = prompt | llm
output = prompt_and_model.invoke(
    {
        "query": f'output the Extracted text, only the english text from the given text: {email_df["subject"][0]}'
    }
)
out = parser.invoke(output)

### Spam check

#### Using POE wrapper for LLMs

In [ ]:
# import time
# from poe_api_wrapper import PoeApi

# token = "FcCAI0DhdVFzH2wL01XC3w=="
# client = PoeApi(token)

# bot = "gpt3_5"

# def ask_poe_gpt(prompt, subject):
#     for chunk in client.send_message(bot, prompt.replace("{subject}", subject)):
#         pass
#     time.sleep(2)
#     client.delete_chat(bot, del_all=True)
#     return chunk["text"]

#### Ollama use-case

In [ ]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

model = "gemma:7b"
# model = "phi:latest"
llm = Ollama(
    model=model,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

In [ ]:
prompt = """
## INPUT MAIL
    Subject: {subject}
    Email Body: {body}
    Email Sender: {sender}

## CONTEXT: 
    You are given an INPUT MAIL that contains the subject and email body and the sender of the email to you.
    You are the look at the contents of the mail and its meta data and infer data from the input so that you look for
    offers, job requests, bank related mails and Urgency of the email. You also need to look at if there are any trivial info in the mail.
    Also pay close attention to the sender email address. Weird looking emails with incorrect spelling and numbers may be spam

## TASK: Based on the above CONTEXT, classify the email as "spam" or "not spam"

## OUTPUT FORMAT

Only output the following output format and keep output verbose
    category: your answer
"""

In [ ]:
# prompt = """
# ## INPUT
# Subject: {subject}
# Email Body: {body}

# ## TASK
# Classify the given mail as "spam" or "not_spam":

# Context: the mail is received by the user Sajid Mashroor
# Extract relevant keywords from the text your reference but you only finally have to classify the given mail as "spam" or "not_spam":

# ## OUTPUT FORMAT
# category: // indicating the classified category i.e., "spam" or "not_spam"
# """

In [ ]:
summary = llm(
    prompt.replace("{subject}", email_df["subject"][5]).replace(
        "{body}", email_df["body"][5]
    )
)
# response = llm(
#     f'Your task is to classify the given summary as "spam" or "not_spam" Based on below summary:\n{summary}'
# )

In [ ]:
# def chunked_body(beta, body):
#     split_idx = int(beta * len(body))
#     return f"{body[:split_idx]}\n{body[-split_idx:]}"

# beta = 0.3

# print(chunked_body(beta, temp))

In [ ]:
response = llm(
    prompt.replace("{subject}", email_df["subject"][5]).replace(
        "{body}", email_df["body"][5]
    )
)

In [8]:
is_spam = """
Given the mail content below; classify the content as spam or important.
The user of the mail is Sajid Mashroor. Whether he should delete the mail or keep it, based on the content.

Answer in yes or no only. Yes meaning "Spam" and No meaning "Important"

{text}
"""